In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold

In [0]:
%sql

CREATE OR REPLACE TABLE gold.full_info AS
SELECT st.sensor_id, s.value, s.pollutant, s.unit, s.datetimeFrom as date, st.station_id, st.name,  st.latitude, st.longitude
FROM silver.sensors as s
INNER JOIN silver.station as st ON s.id = st.sensor_id



In [0]:
%sql

CREATE OR REPLACE TABLE gold.full_info 
PARTITIONED BY (name)
SELECT st.sensor_id, s.value, s.pollutant, s.unit, s.datetimeFrom as date, st.station_id, st.name,  st.latitude, st.longitude
FROM silver.sensors as s
INNER JOIN silver.station as st ON s.id = st.sensor_id

In [0]:
%sql

optimize gold.full_info

In [0]:
%sql
select *
from gold.full_info

In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN hour int



In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN day int

In [0]:
%sql

ALTER TABLE gold.full_info
ADD COLUMN month int

In [0]:
%sql
UPDATE gold.full_info
SET day = EXTRACT(DAY FROM cast(date as timestamp)),
    hour = EXTRACT(HOUR FROM cast(date as timestamp)),
    month = EXTRACT(MONTH FROM cast(date as timestamp));

select *
from gold.full_info

In [0]:
%sql
Select *
from gold.full_info
order by month, day, hour


In [0]:
%sql
CREATE OR REPLACE VIEW gold.distribution AS
SELECT avg(value) as total_value,pollutant, name
FROM gold.full_info
WHERE pollutant not in ('relativehumidity', 'um003', 'temperature'  )  
Group by pollutant, name

In [0]:
%sql
CREATE OR REPLACE VIEW gold.timeline AS
SELECT AVG(value) as value,  name, pollutant, date_trunc('DAY', date) AS date, latitude, longitude

FROM gold.full_info
WHERE pollutant not in ('relativehumidity', 'um003', 'temperature'  )
Group by name, pollutant,  date, latitude, longitude
order by name, pollutant, date, latitude, longitude


In [0]:
%sql
CREATE OR REPLACE VIEW gold.aqi_asset AS
SELECT
  name,
  pollutant,
  value,
  date, latitude, longitude,
  CASE
    WHEN pollutant = 'pm25' THEN
      CASE
        WHEN value BETWEEN 0 AND 12.0 THEN 0
        WHEN value BETWEEN 12.1 AND 35.4 THEN 12.1
        WHEN value BETWEEN 35.5 AND 55.4 THEN 35.5
        WHEN value BETWEEN 55.5 AND 150.4 THEN 55.5
        WHEN value BETWEEN 150.5 AND 250.4 THEN 150.5
        WHEN value BETWEEN 250.5 AND 350.4 THEN 250.5
        WHEN value BETWEEN 350.5 AND 500.4 THEN 350.5
      END
    WHEN pollutant = 'pm10' THEN
    case
        WHEN value BETWEEN 0 AND 54 THEN 0
        WHEN value BETWEEN 55 AND 154 THEN 55
        WHEN value BETWEEN 155 AND 254 THEN 155
        WHEN value BETWEEN 255 AND 354 THEN 255
        WHEN value BETWEEN 355 AND 424 THEN 355
        WHEN value BETWEEN 425 AND 504 THEN 425
        WHEN value BETWEEN 505 AND 604 THEN 505
      END
  END AS min_b,
  CASE
    WHEN pollutant = 'pm25' THEN
      CASE
        WHEN value BETWEEN 0 AND 12.0 THEN 12.0
        WHEN value BETWEEN 12.1 AND 35.4 THEN 35.4
        WHEN value BETWEEN 35.5 AND 55.4 THEN 55.4
        WHEN value BETWEEN 55.5 AND 150.4 THEN 150.4
        WHEN value BETWEEN 150.5 AND 250.4 THEN 250.4
        WHEN value BETWEEN 250.5 AND 350.4 THEN 350.4
        WHEN value BETWEEN 350.5 AND 500.4 THEN 500.4
      END
    WHEN pollutant = 'pm10' THEN
      CASE
        WHEN value BETWEEN 0 AND 54 THEN 54
        WHEN value BETWEEN 55 AND 154 THEN 154
        WHEN value BETWEEN 155 AND 254 THEN 254
        WHEN value BETWEEN 255 AND 354 THEN 354
        WHEN value BETWEEN 355 AND 424 THEN 424
        WHEN value BETWEEN 425 AND 504 THEN 504
        WHEN value BETWEEN 505 AND 604 THEN 604
      END
  END AS max_b,
  CASE
    WHEN pollutant = 'pm25' THEN
      CASE
        WHEN value BETWEEN 0 AND 12.0 THEN 0
        WHEN value BETWEEN 12.1 AND 35.4 THEN 51
        WHEN value BETWEEN 35.5 AND 55.4 THEN 101
        WHEN value BETWEEN 55.5 AND 150.4 THEN 151
        WHEN value BETWEEN 150.5 AND 250.4 THEN 201
        WHEN value BETWEEN 250.5 AND 350.4 THEN 301
        WHEN value BETWEEN 350.5 AND 500.4 THEN 401
      END
    WHEN pollutant = 'pm10' THEN
      CASE
        WHEN value BETWEEN 0 AND 54 THEN 0
        WHEN value BETWEEN 55 AND 154 THEN 51
        WHEN value BETWEEN 155 AND 254 THEN 101
        WHEN value BETWEEN 255 AND 354 THEN 151
        WHEN value BETWEEN 355 AND 424 THEN 201
        WHEN value BETWEEN 425 AND 504 THEN 301
        WHEN value BETWEEN 505 AND 604 THEN 401
      END
  END AS aqi_min,
  CASE
    WHEN pollutant = 'pm25' THEN
      CASE
        WHEN value BETWEEN 0 AND 12.0 THEN 50
        WHEN value BETWEEN 12.1 AND 35.4 THEN 100
        WHEN value BETWEEN 35.5 AND 55.4 THEN 150
        WHEN value BETWEEN 55.5 AND 150.4 THEN 200
        WHEN value BETWEEN 150.5 AND 250.4 THEN 300
        WHEN value BETWEEN 250.5 AND 350.4 THEN 400
        WHEN value BETWEEN 350.5 AND 500.4 THEN 500
      END
        WHEN pollutant = 'pm10' THEN
      CASE
        WHEN value BETWEEN 0 AND 54 THEN 50
        WHEN value BETWEEN 55 AND 154 THEN 100
        WHEN value BETWEEN 155 AND 254 THEN 150
        WHEN value BETWEEN 255 AND 354 THEN 200
        WHEN value BETWEEN 355 AND 424 THEN 300
        WHEN value BETWEEN 425 AND 504 THEN 400
        WHEN value BETWEEN 505 AND 604 THEN 500
      END
  END AS aqi_max

  
FROM gold.timeline
where pollutant in ('pm25', 'pm10')

In [0]:
%sql
select *
from gold.aqi_asset
LIMIT 5

In [0]:
%sql
CREATE OR REPLACE VIEW gold.aqi AS
select ROUND((((aqi_max - aqi_min) / (max_b - min_b)) * (value - min_b) + aqi_min)) as aqi, name, pollutant, value, date, max_b, min_b, aqi_max, aqi_min,  latitude, longitude
from gold.aqi_asset
group by name, pollutant, value, date, max_b, min_b, aqi_max, aqi_min,  latitude, longitude

In [0]:
%sql
select *
from gold.aqi
order by date, name
LIMIT 6

In [0]:
%sql
select *
from gold.full_info
where date_trunc('DAY', date) = '2024-12-31T00:00:00.000+00:00' and name = 'Cologne'

In [0]:
%sql
CREATE OR REPLACE VIEW gold.final_aqi AS
SELECT max(aqi) as aqi, name, date, latitude, longitude
FROM gold.aqi
GROUP BY name, date, latitude, longitude
;


In [0]:
%sql
select *
from gold.final_aqi
order by date, name
LIMIT 6